In [1]:
!pip install simple_salesforce
!pip install pymysql
!pip install sqlalchemy

In [2]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [4]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [5]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Prepare ETL for the Course data

In [78]:
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(30)

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0
5,6,CIS-438,Database Administration,5,0
6,7,CIS-430,Business Systems Programming I,5,0
7,8,CIS-435,Business Systems Programming II,5,0


In [79]:
course_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'CourseName':'Course_Name__c',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

,ID_Course,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [80]:
course_data_df = course_data_df[['Course_Code__c', 'Course_Name__c', 'Credit_Hours__c', 'Boot_Camp_Course__c']]
course_data_df.head()

,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,BC-WEBDEV,Full Stack Web Development,12,1
2,BC-UIUX,User Interface/User Experience,12,1
3,CIS-349,Introduction to Databases,5,0
4,CIS-405,Database Programming,5,0


In [81]:
course_data_records = course_data_df.to_dict('records')
course_data_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Course_Name__c': 'Data Visualization and Analytics',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-WEBDEV',
  'Course_Name__c': 'Full Stack Web Development',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-UIUX',
  'Course_Name__c': 'User Interface/User Experience',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'CIS-349',
  'Course_Name__c': 'Introduction to Databases',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-405',
  'Course_Name__c': 'Database Programming',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-438',
  'Course_Name__c': 'Database Administration',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-430',
  'Course_Name__c': 'Business Systems Programming I',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-435',
  'Course_Name__c':

In [82]:
for rec in course_data_records:

    record = {
        'Course_Code__c': rec['Course_Code__c'],
        'Course_Name__c': rec['Course_Name__c'],
        'Credit_Hours__c': rec['Credit_Hours__c'],
        'Boot_Camp_Course__c': rec['Boot_Camp_Course__c'],
    }
    
    try:
        sf.Course__c.create(record)
    except Exception as e:
        print(e)

In [44]:
# Bulk 
#sf.bulk.Course__c.insert(course_data_records)

## Create Course Lookup Table
You will use this later to crosswalk the course code with the primary key from the `Course` table

It is important to note that we will be querying **Salesforce** to retrieve the record IDs 

In [83]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Course_Code__c, Name FROM Course__c")
for row in data:
    rec = {
        'ID_Course__c': row['Name'], # this is a critical line of code
        'Course_Code__c': row['Course_Code__c']
    }
    course_lookup_list.append(rec)
    
course_lookup_list

[{'ID_Course__c': 'a004x000003VP4M', 'Course_Code__c': 'CIS-430'},
 {'ID_Course__c': 'a004x000003VP4C', 'Course_Code__c': 'CIS-405'},
 {'ID_Course__c': 'a004x000003VP3x', 'Course_Code__c': 'BC-WEBDEV'},
 {'ID_Course__c': 'a004x000003VP4H', 'Course_Code__c': 'CIS-438'},
 {'ID_Course__c': 'a004x000003VP42', 'Course_Code__c': 'BC-UIUX'},
 {'ID_Course__c': 'a004x000003VP4R', 'Course_Code__c': 'CIS-435'},
 {'ID_Course__c': 'a004x000003VP47', 'Course_Code__c': 'CIS-349'},
 {'ID_Course__c': 'a004x000003VP3s', 'Course_Code__c': 'BC-DATAVIZ'}]

In [84]:
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

,ID_Course__c,Course_Code__c
0,a004x000003VP4M,CIS-430
1,a004x000003VP4C,CIS-405
2,a004x000003VP3x,BC-WEBDEV
3,a004x000003VP4H,CIS-438
4,a004x000003VP42,BC-UIUX
5,a004x000003VP4R,CIS-435
6,a004x000003VP47,CIS-349
7,a004x000003VP3s,BC-DATAVIZ


In [85]:
# Query the `Class` table from MySQL
query = '''
    SELECT 
        co.CourseCode,
        cl.*
    FROM 
        class cl
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course

'''

class_data_df = pd.read_sql(query, conn)
class_data_df.head()

,CourseCode,ID_Class,ID_Course,Section,StartDate,EndDate
0,BC-DATAVIZ,1,1,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [86]:
class_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'Section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    'CourseCode':'Course_Code__c'
}, inplace=True)


class_data_df = class_data_df[['Course_Code__c', 'Section__c', 'Start_Date__c', 'End_Date__c']]
class_data_df

,Course_Code__c,Section__c,Start_Date__c,End_Date__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07


## Join the Class DataFrame with the Course lookup table
This join is necessary to successfully lookup the foreign key for the Course table 

In [87]:
class_data_df = pd.merge(class_data_df, course_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

class_data_df

,Course_Code__c,Section__c,Start_Date__c,End_Date__c,ID_Course__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003VP3s
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a004x000003VP3s
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07,a004x000003VP3x
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07,a004x000003VP42


In [88]:
class_data_df
class_data_df['Start_Date__c'] = pd.to_datetime(class_data_df['Start_Date__c']).dt.date
class_data_df['End_Date__c'] = pd.to_datetime(class_data_df['End_Date__c']).dt.date

class_data_df.head()


,Course_Code__c,Section__c,Start_Date__c,End_Date__c,ID_Course__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003VP3s
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a004x000003VP3s
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07,a004x000003VP3x
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07,a004x000003VP42


In [89]:
class_data_records = class_data_df.to_dict(orient='records')
class_data_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': datetime.date(2020, 3, 14),
  'ID_Course__c': 'a004x000003VP3s'},
 {'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWDC201805DATA3',
  'Start_Date__c': datetime.date(2018, 5, 15),
  'End_Date__c': datetime.date(2018, 11, 8),
  'ID_Course__c': 'a004x000003VP3s'},
 {'Course_Code__c': 'BC-WEBDEV',
  'Section__c': 'GWARL201905WEB3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7),
  'ID_Course__c': 'a004x000003VP3x'},
 {'Course_Code__c': 'BC-UIUX',
  'Section__c': 'GWARL201905UIUX3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7),
  'ID_Course__c': 'a004x000003VP42'}]

## Insert `Class` Records into Salesforce

In [90]:
for rec in class_data_records:
 
    record = {
        'ID_Course__c': rec['ID_Course__c'],
        'Section__c': rec['Section__c'],
        'Start_Date__c': str(rec['Start_Date__c']),
        'End_Date__c': str(rec['End_Date__c']),
    }
    
    try:
        sf.Class__C.create(record)
    except Exception as e:
        print(e)

# Prepare ETL for Student Data

In [144]:
student_data_df = pd.read_sql("Select * from student", conn)
student_data_df.head()

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [145]:
student_data_df.rename(columns={
    'ID_Student':'ID_Student__c',
    'StudentID': 'StudentID__c',
    'FirstName':'FirstName__c',
    'MiddleName': 'MiddleName__c',
    'LastName': 'LastName__c',
    'BirthDate': 'Birthdate__c',
    'Gender': 'Gender__c'
}, inplace=True)
student_data_df.head()

,ID_Student__c,StudentID__c,LastName__c,FirstName__c,MiddleName__c,Birthdate__c,Gender__c
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [146]:
student_data_records = student_data_df.to_dict('records')
student_data_records

[{'ID_Student__c': 33,
  'StudentID__c': '25004961',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Heather',
  'MiddleName__c': 'Alice',
  'Birthdate__c': None,
  'Gender__c': 'F'},
 {'ID_Student__c': 34,
  'StudentID__c': '25003514',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Lana',
  'MiddleName__c': 'Cecille',
  'Birthdate__c': None,
  'Gender__c': 'F'},
 {'ID_Student__c': 35,
  'StudentID__c': '25005833',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Jessica',
  'MiddleName__c': 'Dorothy',
  'Birthdate__c': None,
  'Gender__c': 'F'},
 {'ID_Student__c': 36,
  'StudentID__c': '25002589',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Kimberly',
  'MiddleName__c': 'Genevieve',
  'Birthdate__c': None,
  'Gender__c': 'F'},
 {'ID_Student__c': 37,
  'StudentID__c': '25007185',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Katherine',
  'MiddleName__c': 'Lynnette',
  'Birthdate__c': None,
  'Gender__c': 'F'},
 {'ID_Student__c': 38,
  'StudentID__c': '25006014',
  'LastName__c':

In [147]:
for student in student_data_records:
    
    record = {
        'ID_Student__c': student['ID_Student__c'],
        'StudentID__c': student['StudentID__c'],
        'LastName__c': student['LastName__c'],
        'FirstName__c': student['FirstName__c'],
        'MiddleName__c': student['MiddleName__c'],
        'Birthdate__c': student['Birthdate__c'],
        'Gender__c': student['Gender__c']
    }
    
    try:
        sf.Student__c.create(record)
    except Exception as e:
        print(e)

# Prepare Class Participant ETL

#1 Create Class  Lookup Table

In [122]:
class_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Section__c, Name FROM Class__c")
for row in data:
    rec = {
        'ID_Class__c': row['Name'], # this is a critical line of code
        'Section__c': row['Section__c']
    }
    class_lookup_list.append(rec)
    
class_lookup_list

[{'ID_Class__c': 'a014x000008WKRH', 'Section__c': 'GWARL201905WEB3'},
 {'ID_Class__c': 'a014x000008WKRC', 'Section__c': 'GWDC201805DATA3'},
 {'ID_Class__c': 'a014x000008WKRM', 'Section__c': 'GWARL201905UIUX3'},
 {'ID_Class__c': 'a014x000008WKR7', 'Section__c': 'GWU-ARL-DATA-PT-09-0'}]

In [123]:
class_lookup_df = pd.DataFrame(class_lookup_list)
class_lookup_df

,ID_Class__c,Section__c
0,a014x000008WKRH,GWARL201905WEB3
1,a014x000008WKRC,GWDC201805DATA3
2,a014x000008WKRM,GWARL201905UIUX3
3,a014x000008WKR7,GWU-ARL-DATA-PT-09-0


In [124]:
#Query the 'Class Participants' table from MySql

cpquery = '''
    SELECT 
        cl.Section,
        cp.*
    FROM 
        class cl
        INNER JOIN classparticipant cp
        ON cl.ID_Class = cp.ID_Class

'''

cp_data_df = pd.read_sql(cpquery, conn)
cp_data_df.head()




,Section,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,GWU-ARL-DATA-PT-09-0,1,33,1,2020-09-16,None
1,GWU-ARL-DATA-PT-09-0,2,34,1,2020-09-16,None
2,GWU-ARL-DATA-PT-09-0,3,35,1,2020-09-16,None
3,GWU-ARL-DATA-PT-09-0,4,62,1,2020-09-16,None
4,GWU-ARL-DATA-PT-09-0,5,36,1,2020-09-16,None


In [125]:
cp_data_df.rename(columns={
    'Section': 'Section__c',
    'ID_ClassParticipant': 'ID_ClassParticipant__c',
    'ID_Student': 'ID_Student__c',
    'StartDate': 'StartDate__c',
    'EndDate': 'EndDate__c'
}, inplace=True)

#cp_data_df = cp_data_df[['ID_ClassParticipant__c', 'ID_Class__c','ID_Student__c','StartDate__c','EndDate__c']]
cp_data_df.head()

,Section__c,ID_ClassParticipant__c,ID_Student__c,ID_Class,StartDate__c,EndDate__c
0,GWU-ARL-DATA-PT-09-0,1,33,1,2020-09-16,None
1,GWU-ARL-DATA-PT-09-0,2,34,1,2020-09-16,None
2,GWU-ARL-DATA-PT-09-0,3,35,1,2020-09-16,None
3,GWU-ARL-DATA-PT-09-0,4,62,1,2020-09-16,None
4,GWU-ARL-DATA-PT-09-0,5,36,1,2020-09-16,None


In [126]:
#Join the classparticipants dataframe with the Class Lookup table

cp_data_df = pd.merge(cp_data_df, class_lookup_df, how='left')

#choose columns to include
cp_data_df = cp_data_df[['ID_ClassParticipant__c','ID_Student__c','StartDate__c','EndDate__c','ID_Class__c']]
cp_data_df.head()

,ID_ClassParticipant__c,ID_Student__c,StartDate__c,EndDate__c,ID_Class__c
0,1,33,2020-09-16,None,a014x000008WKR7
1,2,34,2020-09-16,None,a014x000008WKR7
2,3,35,2020-09-16,None,a014x000008WKR7
3,4,62,2020-09-16,None,a014x000008WKR7
4,5,36,2020-09-16,None,a014x000008WKR7


In [140]:
#insert class participants into Salesforce
cpclass_data_records = cp_data_df.to_dict(orient='records')
cpclass_data_records

[{'ID_ClassParticipant__c': 1,
  'ID_Student__c': 33,
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None,
  'ID_Class__c': 'a014x000008WKR7'},
 {'ID_ClassParticipant__c': 2,
  'ID_Student__c': 34,
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None,
  'ID_Class__c': 'a014x000008WKR7'},
 {'ID_ClassParticipant__c': 3,
  'ID_Student__c': 35,
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None,
  'ID_Class__c': 'a014x000008WKR7'},
 {'ID_ClassParticipant__c': 4,
  'ID_Student__c': 62,
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None,
  'ID_Class__c': 'a014x000008WKR7'},
 {'ID_ClassParticipant__c': 5,
  'ID_Student__c': 36,
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None,
  'ID_Class__c': 'a014x000008WKR7'},
 {'ID_ClassParticipant__c': 6,
  'ID_Student__c': 37,
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None,
  'ID_Class__c': 'a014x000008WKR7'},
 {'ID_ClassParticipant__c': 7,
  'ID_Student__

#2 Create the Student Lookup table

In [154]:
student_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT StudentID__c, Name FROM Student__c")
for row in data:
    rec = {
        'Student_ID__c': row['Name'], # this is a critical line of code
        'StudentID__c': row['StudentID__c']
    }
    student_lookup_list.append(rec)
    
student_lookup_list

[{'Student_ID__c': 'a054x0000010eDE', 'StudentID__c': '25006348'},
 {'Student_ID__c': 'a054x0000010eDJ', 'StudentID__c': '25002555'},
 {'Student_ID__c': 'a054x0000010eBm', 'StudentID__c': '25004961'},
 {'Student_ID__c': 'a054x0000010eCp', 'StudentID__c': '25005602'},
 {'Student_ID__c': 'a054x0000010eE7', 'StudentID__c': '25007334'},
 {'Student_ID__c': 'a054x0000010eCV', 'StudentID__c': '25002056'},
 {'Student_ID__c': 'a054x0000010eCa', 'StudentID__c': '25004152'},
 {'Student_ID__c': 'a054x0000010eD9', 'StudentID__c': '25005284'},
 {'Student_ID__c': 'a054x0000010eDO', 'StudentID__c': '25005250'},
 {'Student_ID__c': 'a054x0000010eDY', 'StudentID__c': '25003773'},
 {'Student_ID__c': 'a054x0000010eE2', 'StudentID__c': '25007333'},
 {'Student_ID__c': 'a054x0000010eCL', 'StudentID__c': '25003778'},
 {'Student_ID__c': 'a054x0000010eCQ', 'StudentID__c': '25003605'},
 {'Student_ID__c': 'a054x0000010eCu', 'StudentID__c': '25006805'},
 {'Student_ID__c': 'a054x0000010eDT', 'StudentID__c': '2500232

In [155]:
student_lookup_df = pd.DataFrame(student_lookup_list)

student_lookup_df.head()

,Student_ID__c,StudentID__c
0,a054x0000010eDE,25006348
1,a054x0000010eDJ,25002555
2,a054x0000010eBm,25004961
3,a054x0000010eCp,25005602
4,a054x0000010eE7,25007334


In [156]:
#Query the 'Students' table from MySql

stquery = '''
    SELECT 
        st.StudentID,
        cp.*
    FROM 
        student st
        INNER JOIN classparticipant cp
        ON cp.ID_Student = st.ID_Student
        
'''

cp1_data_df = pd.read_sql(stquery, conn)
cp1_data_df.head()

,StudentID,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,25004961,1,33,1,2020-09-16,None
1,25003514,2,34,1,2020-09-16,None
2,25005833,3,35,1,2020-09-16,None
3,25007334,4,62,1,2020-09-16,None
4,25002589,5,36,1,2020-09-16,None


In [158]:
cp1_data_df.rename(columns={
    'StudentID':'StudentID__c',
    'ID_Student': 'ID_Student__c',
    'StartDate': 'StartDate__c',
    'EndDate': 'EndDate__c',
    'ID_ClassParticipant': 'ID_ClassParticipant__c',
    'ID_Class': 'ID_Class__c'  
}, inplace=True)

cp1_data_df.head() 

,StudentID__c,ID_ClassParticipant__c,ID_Student__c,ID_Class__c,StartDate__c,EndDate__c
0,25004961,1,33,1,2020-09-16,None
1,25003514,2,34,1,2020-09-16,None
2,25005833,3,35,1,2020-09-16,None
3,25007334,4,62,1,2020-09-16,None
4,25002589,5,36,1,2020-09-16,None


In [175]:
#Join the students/classparticipants dataframe with the student Lookup table

cp1_data_df = pd.merge(cp1_data_df, student_lookup_df, how='left')

#choose columns to include
cp1_data_df = cp1_data_df[['StudentID__c','ID_Student__c','Student_ID__c']]
cp1_data_df.head()

,StudentID__c,ID_Student__c,Student_ID__c
0,25004961,33,a054x0000010eBm
1,25003514,34,a054x0000010eBr
2,25005833,35,a054x0000010eBw
3,25007334,62,a054x0000010eE7
4,25002589,36,a054x0000010eC1


In [176]:
cp_data_df.head()

,ID_ClassParticipant__c,ID_Student__c,StartDate__c,EndDate__c,ID_Class__c
0,1,33,2020-09-16,None,a014x000008WKR7
1,2,34,2020-09-16,None,a014x000008WKR7
2,3,35,2020-09-16,None,a014x000008WKR7
3,4,62,2020-09-16,None,a014x000008WKR7
4,5,36,2020-09-16,None,a014x000008WKR7


In [177]:
#Join the students/classparticipants w class/classparticipants

newcp_data_df = pd.merge(cp1_data_df, cp_data_df, on='ID_Student__c')
newcp_data_df.head()

,StudentID__c,ID_Student__c,Student_ID__c,ID_ClassParticipant__c,StartDate__c,EndDate__c,ID_Class__c
0,25004961,33,a054x0000010eBm,1,2020-09-16,None,a014x000008WKR7
1,25003514,34,a054x0000010eBr,2,2020-09-16,None,a014x000008WKR7
2,25005833,35,a054x0000010eBw,3,2020-09-16,None,a014x000008WKR7
3,25007334,62,a054x0000010eE7,4,2020-09-16,None,a014x000008WKR7
4,25002589,36,a054x0000010eC1,5,2020-09-16,None,a014x000008WKR7


In [178]:
cp_records = newcp_data_df.to_dict(orient='records')
cp_records

[{'StudentID__c': '25004961',
  'ID_Student__c': 33,
  'Student_ID__c': 'a054x0000010eBm',
  'ID_ClassParticipant__c': 1,
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None,
  'ID_Class__c': 'a014x000008WKR7'},
 {'StudentID__c': '25003514',
  'ID_Student__c': 34,
  'Student_ID__c': 'a054x0000010eBr',
  'ID_ClassParticipant__c': 2,
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None,
  'ID_Class__c': 'a014x000008WKR7'},
 {'StudentID__c': '25005833',
  'ID_Student__c': 35,
  'Student_ID__c': 'a054x0000010eBw',
  'ID_ClassParticipant__c': 3,
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None,
  'ID_Class__c': 'a014x000008WKR7'},
 {'StudentID__c': '25007334',
  'ID_Student__c': 62,
  'Student_ID__c': 'a054x0000010eE7',
  'ID_ClassParticipant__c': 4,
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None,
  'ID_Class__c': 'a014x000008WKR7'},
 {'StudentID__c': '25002589',
  'ID_Student__c': 36,
  'Student_ID__c': 'a054x0000010eC1',
 

In [181]:
#insert records into classparticipant
for rec in cp1_records:
 
    record = {
        'Student_ID__c': rec['Student_ID__c'],
        'ID_Student__c': rec['ID_Student__c'],
        'StartDate__c': str(rec['StartDate__c']),
        'EndDate__c': str(rec['EndDate__c']),
        'ID_Class__c':rec['ID_Class__c']
    }
    
    try:
        sf.ClassParticipant__C.create(record)
    except Exception as e:
        print(e)

Malformed request https://gwu4-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/ClassParticipant__C/. Response content: [{'message': "No such column 'Student_ID__c' on sobject of type ClassParticipant__c", 'errorCode': 'INVALID_FIELD'}]
Malformed request https://gwu4-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/ClassParticipant__C/. Response content: [{'message': "No such column 'Student_ID__c' on sobject of type ClassParticipant__c", 'errorCode': 'INVALID_FIELD'}]
Malformed request https://gwu4-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/ClassParticipant__C/. Response content: [{'message': "No such column 'Student_ID__c' on sobject of type ClassParticipant__c", 'errorCode': 'INVALID_FIELD'}]
Malformed request https://gwu4-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/ClassParticipant__C/. Response content: [{'message': "No such column 'Student_ID__c' on sobject of type ClassParticipant__c", 'errorCode': 'INVALID_FIELD'}]
Malformed request https://gwu4-d

## Example of Deleting Records

Select the IDs of the records first and then process the results.

Ultimately, you want a list of IDs in the end.


In [142]:
class_records = sf.query("SELECT Id FROM Student__c")
recs_to_delete = [{'Id': r['Id']} for r in class_records['records']]
recs_to_delete

[{'Id': 'a054x0000010e9MAAQ'},
 {'Id': 'a054x0000010e9RAAQ'},
 {'Id': 'a054x0000010e9WAAQ'},
 {'Id': 'a054x0000010e9bAAA'},
 {'Id': 'a054x0000010e9gAAA'},
 {'Id': 'a054x0000010e9lAAA'},
 {'Id': 'a054x0000010e9qAAA'},
 {'Id': 'a054x0000010e9vAAA'},
 {'Id': 'a054x0000010eA0AAI'},
 {'Id': 'a054x0000010eA5AAI'},
 {'Id': 'a054x0000010eAAAAY'},
 {'Id': 'a054x0000010eAFAAY'},
 {'Id': 'a054x0000010eAKAAY'},
 {'Id': 'a054x0000010eAPAAY'},
 {'Id': 'a054x0000010eAUAAY'},
 {'Id': 'a054x0000010eAZAAY'},
 {'Id': 'a054x0000010eAeAAI'},
 {'Id': 'a054x0000010eAjAAI'},
 {'Id': 'a054x0000010eAoAAI'},
 {'Id': 'a054x0000010eAtAAI'},
 {'Id': 'a054x0000010eAyAAI'},
 {'Id': 'a054x0000010eB3AAI'},
 {'Id': 'a054x0000010eB8AAI'},
 {'Id': 'a054x0000010eBDAAY'},
 {'Id': 'a054x0000010eBIAAY'},
 {'Id': 'a054x0000010eBNAAY'},
 {'Id': 'a054x0000010eBSAAY'},
 {'Id': 'a054x0000010eBXAAY'},
 {'Id': 'a054x0000010eBcAAI'},
 {'Id': 'a054x0000010eBhAAI'}]

In [ ]:
#sf.bulk.Course__c.delete(recs_to_delete)

In [143]:
for rec in recs_to_delete:
    try:
        sf.Student__c.delete(rec['Id'])
    except Exception as e:
        print(e)